# Нейрон

## Математическая модель нейрона

Математический нейрон представляет собой сумматор, единственный выход которого определяется через его входы в матрицу весов

$$y = f(u)$$

$$u = \sum_{i=1}^n w_{i}x_{i} + b$$

Где $x_{i}$ - сигналы на входах нейрона, $w_{i}$ - веса входов, $f(u)$ - функция активации, $b$ - смещение.

**Функция активации** определяет зависимость выхода нейрона от суммы сигналов на входах. Обычно она является монотонно возрастающей и имеет область значений [-1,1] или [0,1] (например, ReLU, tanh, sigmoid).

Что именно нейрон делает с входными данными (т.е. результат работы нейрона) определяется его **весами**, которые являются матрицей чисел. В данном случае _обучение_ подразумевает поиск набора значений весов всех нейронов, при которых сеть будет правильно отображать тестовые входные данные в соответствующие им результаты. Глубокая нейронная сеть может содержать миллионы параметров, корректировка каждого из них - сложная задача.

Чтобы управлять результатом работы нейронной сети, нужно измерить насколько этот результат далек от ожидаемого. Эту задачу решает **функция потер** или **целевая функция**. Целевая функция принимает решение нейронной сети и истинное значение (которое должна вернуть обученная сеть) и вычисляет расстояние между ними, отражающее насколько хорошо сеть справилась с данным примером.

Данная оценка используется для корректировки значений весов с целью уменьшения потерь в данном примере. Корректировкой занимается *оптимизатор*, реализующий **алгоритм обратного распространения ошибки**. Изначально весам присваиваются случайные значения, оценка потерь очень высока. С каждой итерацией веса корректируются, оценка потерь уменьшается. Сеть с минимальными потерями, возвращающая результаты, близкие к истинным, называется обученной сетью.

## Пример
Рассмотрим конкретную нейронную сеть, которая обучается классификации рукописных цифр. Нужно реализовать классификацию черно-белых изображений цифр $28\times28$ пикселов по $10$ категориям. Используется набор данных MNIST, который содержит $60 000$ обучающих изображений и $10 000$ контрольных.

In [3]:
from keras.datasets import mnist #загрузка тренировочных и тестовых данных
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [4]:
from keras import models
from keras import layers
#формирование сети из 2 полносвязных слоев
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

In [5]:
#настройка параметров компиляции: оптимизатор, функция потерьи метрики
network.compile(optimizer='rmsprop',
loss='categorical_crossentropy',
metrics=['accuracy'])

In [6]:
#преобразование исходных данных
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

In [7]:
#кодирование меток категорий
from keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [8]:
#обучение нейронной сети
>>> network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
60000/60000 [==============================] - 9s 148us/step - loss: 0.2548 - accuracy: 0.9266
Epoch 2/5
60000/60000 [==============================] - 8s 137us/step - loss: 0.1037 - accuracy: 0.9695
Epoch 3/5
60000/60000 [==============================] - 8s 139us/step - loss: 0.0683 - accuracy: 0.9795
Epoch 4/5
60000/60000 [==============================] - 8s 132us/step - loss: 0.0495 - accuracy: 0.9847
Epoch 5/5
60000/60000 [==============================] - 8s 129us/step - loss: 0.0377 - accuracy: 0.9887


In [9]:
#проверка обученной сети на тестовом наборе
>>> test_loss, test_acc = network.evaluate(test_images, test_labels)
>>> print('test_acc:', test_acc)

10000/10000 [==============================] - 1s 99us/step
test_acc: 0.9783999919891357


В процессе обучения отображаются потери сети и точность на обучающих данных. На тестовом (контрольном) наборе точность составила $0.978$, что меньше точности обучающих данных $0.989$. Нейронная сеть *переобучилась* на тренировочных данных.

Вы знаете, что эта сеть состоит из цепочки двух слоев, каждый из которых применяет к входным данным несколько операций,которые вовлекают вовлекают веса. Веса (атрибуты слоев) - это место, где накапливаются знания сети. 

`categorical_crossentropy` - функция потерь, которая используется для обучения весов и которую во время обучения сеть стремится свести к минимуму. Снижение потерь достигается за счет *градиентного спуска*.

**Градиент** - это производная на функции с многомерными входными данными; вектор, своим направлением указывающий направление наибольшего возрастания. Рассмотрим входной вектор $X$, матрицу весов $W$ и функцию потерь loss. Можно с их помощью вычислить кандидата $y'$ и определить расстояние (несоответствие) $d$ между кандидатом и целью $y$.
$$y' = X \times W$$
$$d = loss(y',y)$$
Если зафиксировать входные данные, то последнюю функцию можно интерпретировать как зависимость расстояния от веса.
$$d = f(W)$$
Тогда производная функции $f$ в текущем значении $W$ будет матрицей, каждый элемент которой определяет направление и величину изменения в $d$, наблюдаемого при изменении текущего значения. Эта матрица и есть градиент функции $f$.

Как производная функции единственного аргумента интерпритируется, как наклон кривой. Так градиент многомерной функции можно интерпритировать, как вектор, описывающий кривизну в окрестностях текущей точки. Для уменьшения потерь, нужно двигаться в сторону, противоположную градиенту, то есть, стремиться к *минимуму*, этот метод называется **градиентный спуск**.

## Градиентный спуск
Градиентный спуск - это метод нахождения *минимума* функции с помщью движения вдоль градиента.

В теории минимум дифференцируемой функции можно найти следующим образом: найти все точки, где производная обращается в 0, и выяснить в какой из этих точек функция минимальна. Решить такие уравнения не составит труда для 2 или 3 переменных, но в нейронных сетях используются тысячи, а не редко и миллионы весов в сети. Поэтому на практике используется **стохатический градиентный спуск на небольших пакетах**
1. Извлекается пакет обучающих экземпляров $X$ и соответствующих им цулей $y$
2. Сеть обрабатывает пакет $X$ и получает предсказания $y'$
3. Вычисляются потери сети $d$
4. Вычисляется градиент потерь для параметров сети (*обратный проход*)
5. Параметры корректируются на небольшую величину в направлении, противоположном градиенту ($W = W -(step*gradient)$)

"Стохатический" отражает, что каждый пакет данных выбирается случайно. Выбор разумной величины шага $step$ имеет значение. При слишком большом шаге, корректировки будут хаотичными и приведут в случайные точки. При слишком маленьком шаге, потребуется большое количество итераций, так же есть риск застрять в локальном минимуме. 

Вариант алгоритма, приведенный выше, в каждой итерации использует один образец и цель, а не весь пакет данных. Можно использовать на каждом шаге все доступные данные, такая версия алгоритма называется **пакетным стохатическим градиентным спуском**. В таком случае, изменения будут более точными, но потребуют больших затрат.